<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/curso3/ciclo4/3_fastapi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?export=view&id=1o4udU5qVMi_7jDi0XzSspbPC6Hw0ev9o" width="100%">

# **Despliege de Modelos con FastAPI**
---

En este notebook veremos cómo podemos crear APIs de modelos personalizadas con `fastapi` y modelos de `sklearn`. Comenzamos instalando `fastapi`:

In [ ]:
!pip install fastapi

Importamos las librerías necesarias:

In [ ]:
import pandas as pd
from IPython.display import display

## **1. Carga de Datos**
---

En este caso utilizaremos el conjunto de datos [hate speech and offensive language dataset](https://www.kaggle.com/datasets/mrmorj/hate-speech-and-offensive-language-dataset) de Kaggle. Se trata de una colección de textos etiquetados manualmente que se utilizan para detectar el discurso de odio y el lenguaje ofensivo. Los textos se han recopilado de varias fuentes, incluidas redes sociales como Twitter, y contienen una variedad de comentarios y mensajes que han sido etiquetados como ofensivos o no ofensivos.

<center><img src="https://drive.google.com/uc?export=view&id=1juGD2mUOGJ27gVhIGS3zUCFsXA9R67qI" width="80%"></center>

En general, este conjunto de datos es útil para aquellos interesados en la detección automática de discurso de odio y lenguaje ofensivo en línea, se puede utilizar para entrenar modelos de aprendizaje automático para identificar y clasificar este tipo de contenido. Además, el conjunto de datos también puede ser utilizado por investigadores interesados en analizar el uso de lenguaje ofensivo en línea y cómo afecta a las personas y comunidades.

Para este ejemplo utilizaremos una muestra de 8000 documentos del corpus completo. Procedemos a cargarlo:

In [ ]:
data = pd.read_parquet("https://raw.githubusercontent.com/mindlab-unal/mlds6-datasets/main/u4/hate.parquet")
display(data.head())

Este conjunto de datos tiene las siguientes dos columnas:

- `text`: texto de un tweet.
- `label`: indica si hay un discurso de odio 1 o no 0.

## **2. Modelamiento y Evaluación**
---

Vamos a entrenar un modelo para clasificar automáticamente discursos de odio a partir de textos. Comenzamos dividiendo el conjunto de datos en las particiones de entrenamiento y prueba:

In [ ]:
from sklearn.model_selection import train_test_split

Primero dividimos el conjunto de datos:

In [ ]:
corpus_train, corpus_test, labels_train, labels_test = train_test_split(
        data.text, data.label, stratify=data.label, test_size=0.3, random_state=0
        )

Para la clasificación utilizaremos un modelo de bosques aleatorios junto con una representación de tipo TF-IDF, adicionalmente creamos un pipeline para tener un modelo completo:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

Definimos el modelo:

In [ ]:
model = Pipeline([
    ("extractor", TfidfVectorizer(max_df=0.3, max_features=2000)),
    ("clf", RandomForestClassifier(max_depth=5, random_state=0))
    ])

Entrenamos el modelo:

In [ ]:
model.fit(corpus_train, labels_train)

Evaluamos el desempeño del modelo:

In [ ]:
print(model.score(corpus_test, labels_test))

Finalmente, utilizaremos la herramienta `joblib` para guardar el modelo:

In [ ]:
import joblib

Guardamos el modelo:

In [ ]:
joblib.dump(model, "model.joblib")

## **3. FastAPI**
---

FastAPI es un framework moderno y rápido para la construcción de aplicaciones web basado en Python. Es muy popular en la comunidad de Machine Learning porque ofrece una serie de características que lo hacen ideal para crear API RESTful para servicios de aprendizaje automático. En particular, FastAPI se destaca por su velocidad y eficiencia, lo que lo hace ideal para aplicaciones de aprendizaje automático que requieren un procesamiento rápido de solicitudes y respuestas. Además, FastAPI utiliza la tipificación de Python, lo que ayuda a detectar errores en tiempo de compilación y proporciona una mejor documentación y autocompletado en el editor de código.

Otra razón por la que FastAPI es popular en la comunidad de Machine Learning es su facilidad de uso y su capacidad para manejar diferentes tipos de datos. FastAPI se integra fácilmente con bibliotecas populares de aprendizaje automático como `tensorflow`, `pytorch` y `sklearn`, lo que lo hace ideal para crear servicios API RESTful que utilizan modelos de aprendizaje automático. FastAPI tambien proporciona una documentación automática basada en OpenAPI y Swagger, lo que permite a los desarrolladores y usuarios de la API acceder fácilmente a la documentación detallada de la API y sus especificaciones.

Para desplegar un modelo de machine learning con `fastapi` estaremos utilizando el protocolo HTTP tal y como se muestra a continuación:

<img src="https://drive.google.com/uc?export=view&id=1JCy0PSAFDalM0OzqH9kYu3s4iwjs3JJR" width="80%">

En específico, vamos a implementar una función que permita manejar llamadas de tipo **post** (recibe y retorna datos). Para esto, comenzamos definiendo la entrada del API como una clase usando `pydantic` (permite estructurar y validar tipos):

In [ ]:
from pydantic import BaseModel
from typing import List

class ApiInput(BaseModel):
    text: List[str]

Ahora, definimos una clase para la salida del API:

In [ ]:
class ApiOutput(BaseModel):
    is_hate: List[int]

Ahora, veamos cómo es el flujo de trabajo para un API personalizada de machine learning:

<img src="https://drive.google.com/uc?export=view&id=1Qj6Njwl2DlVCkPxTddB-Gen-piQAlL2L" width="80%">

Esta se compone de los siguientes elementos:

- **Cliente**: el cliente hace referencia al usuario o la aplicación que enviará datos y recibirá predicciones.
- **Endpoint**: se trata de una url única del API que nos permite interactuar con un recurso o función determinada, en específico, la predicción de un modelo. En este caso lo manejamos por medio de `fastapi`.
- **Registro del modelo**: se trata del almacenamiento donde quedó guardado el modelo. En este caso es un único archivo, aunque se puede manejar de una forma más precisa con herramientas como `mlflow` o `dvc`.
- **Creación del modelo**: se trata del proceso (normalmente fuera de línea) que entrena un modelo y lo guarda en el **registro**.

Veamos cómo podemos definir el **endpoint** con `fastapi`. Esto se debe generar como un script de _Python_ tal y como se muestra a continuación:

In [ ]:
%%writefile main.py
from fastapi import FastAPI # importamos el API
from pydantic import BaseModel
from typing import List
import joblib # importamos la librería para cargar el modelo

class ApiInput(BaseModel):
    texts: List[str]

class ApiOutput(BaseModel):
    is_hate: List[int]

app = FastAPI() # creamos el api
model = joblib.load("model.joblib") # cargamos el modelo.

@app.post("/hate") # creamos api que permita requests de tipo post.
async def define_sentiment(data: ApiInput) -> ApiOutput:
    predictions = model.predict(data.texts).flatten().tolist() # generamos la predicción
    preds = ApiOutput(is_hate=predictions) # estructuramos la salida del API.
    return preds # retornamos los resultados

## **4. Railway**
---

Vamos a realizar el despliegue de esta API por medio de la plataforma Railway. Se trata de una plataforma en línea que ofrece servicios de alojamiento web, bases de datos y herramientas de desarrollo para crear y desplegar aplicaciones web. Es una plataforma todo en uno que proporciona un entorno de desarrollo simplificado y una infraestructura escalable para los desarrolladores.

<img src="https://drive.google.com/uc?export=view&id=1l0FTVsrZR4XTZj7ZRSyIajmUdwxeF5gi" width="80%">

En cuanto a las aplicaciones de Machine Learning, Railway ofrece algunas características que pueden ser útiles para desplegar APIs de Machine Learning:

- **Integración con GitHub**: Railway se integra perfectamente con GitHub, lo que permite desplegar aplicaciones de Machine Learning directamente desde repositorios.
- **Fácil configuración de variables de entorno**: Railway proporciona una interfaz fácil de usar para configurar variables de entorno para la aplicación, lo que es útil para definir y administrar los parámetros de configuración de los modelos de Machine Learning.
- **Escalabilidad automática**: Railway proporciona una infraestructura escalable y una configuración automática de recursos para asegurar que la aplicación pueda manejar un alto volumen de tráfico y se pueda escalar según sea necesario.

Para comenzar a usar railway debe dirigirse a [este enlace](https://railway.app/) y acceder por medio de su correo:

<img src="https://drive.google.com/uc?export=view&id=1KXmduYiolCjWTHq268QXxqNC0GXh96E_" width="80%">

Por defecto Railway ofrece 2 dolares gratuitos, no obstante, si valida su cuenta puede obtener 5 dolares gratuitos:

<img src="https://drive.google.com/uc?export=view&id=1FdDa4O7K79ON47xqmfqd141Dq_bDMwBx" width="80%">

Para desplegar el API debemos crear un repositorio:

In [ ]:
!mkdir mlapi
!mv main.py model.joblib mlapi/
%cd mlapi/

Inicializamos el repositorio:

In [ ]:
!git config --global user.email "email"
!git config --global user.name "usuario o nombre"
!git config --global init.defaultBranch master
!git init

Adicionalmente, debemos crear el archivo `requirements.txt` con las dependencias del proyecto:

In [ ]:
%%writefile requirements.txt
scikit-learn
fastapi
uvicorn
joblib

Adicional a esto, Railway requiere la creación de un archivo de configuración:

In [ ]:
%%writefile railway.json
{
  "$schema": "https://railway.app/railway.schema.json",
  "build": {
    "builder": "NIXPACKS"
  },
  "deploy": {
    "startCommand": "uvicorn main:app --host 0.0.0.0 --port $PORT",
    "restartPolicyType": "ON_FAILURE",
    "restartPolicyMaxRetries": 10
  }
}

Ahora agregamos los archivos al area de preparación:

In [ ]:
!git add railway.json requirements.txt main.py model.joblib
!git commit -m "Agregamos los archivos del API."

En GitHub, debe crear un repositorio y agregar los siguientes dos campos (puede revisar el material de la unidad 2 si no recuerda el proceso):

In [ ]:
token = ""  # Agregue su token dentro de las comillas.
repo_url = "" # Agregue la url de su repositorio dentro de las comillas.

Ahora, usaremos una expresión regular para reemplazar el token en esta url:

In [ ]:
import re
pat = re.compile(r"(https://)(.*)")

Formateamos la URL:

In [ ]:
import os
match = re.match(pat, repo_url)
url_token = "".join([match.group(1), token, "@", match.group(2)])
os.environ["GITHUB"] = url_token

Finalmente, enlazamos el repositorio local con el nuevo repositorio en Github con el comando `git remote`:

In [ ]:
!git remote add origin $GITHUB

En este punto, las versiones local (Colab) y remoto (Github) son distintas, para subir los cambios podemos usar el comando `git push` especificando la rama que deseamos actualizar:

In [ ]:
!git push origin master

Ahora, desde Railway debemos crear un nuevo proyecto:

<img src="https://drive.google.com/uc?export=view&id=1gAXJbVB1mpmT-W18pfUfUl3u6xsD2CQ-" width="80%">

El proyecto debe ser creado desde GitHub:

<img src="https://drive.google.com/uc?export=view&id=1XHdYwVhKmH3fnLLeEcdaBa7QOKRHP_ks" width="80%">

Ahora, debe seleccionar el repositorio que acaba de actualizar:

<img src="https://drive.google.com/uc?export=view&id=1QGyztFCLuKR7ah8dtlUNiywP97NY5REe" width="80%">

Con esto, podrá ver que el API comienza a desplegar dentro de la plataforma:

<img src="https://drive.google.com/uc?export=view&id=1WkaMtPbRFSLACYmQosSnH6sJTKuMfSKi" width="80%">

Finalmente, cuando el API esté desplegado debe irse a la sección **Settings** y dar click sobre el botón **Generate Domain**, esto generará una url que debe pegar en la siguiente variable (debe validar que la url contenga `"https://"` al inicio, en caso contrario debe agregarlo).

In [ ]:
model_url = "https://mlapi-production-272c.up.railway.app" # Agregue acá la url de railway

Ahora podemos validar que el API funcione correctamente, vamos a consumirlo con la librería `requests`:

In [ ]:
import requests

Veamos el resultado para un texto con odio:

In [ ]:
r = requests.post(os.path.join(model_url, "hate"), json={"texts": ["you are so dumb and a stupid and ignorant person"]})
print(r.json()) # hate

Ahora para un texto normal:

In [ ]:
r = requests.post(os.path.join(model_url, "hate"), json={"texts": ["You are so nice and humble"]})
print(r.json()) # normal

## Recursos Adicionales
---

Los siguientes enlaces corresponden a sitios donde encontrará información muy útil para profundizar en los temas vistos en este notebook:

- [MLFlow Models](https://mlflow.org/docs/latest/models.html).
- [MLFlow Model Serving](https://towardsdatascience.com/mlflow-model-serving-bcd936d59052).
- [Qué es un API de Rest?](https://www.redhat.com/es/topics/api/what-is-a-rest-api).

## Créditos
---

**Profesor**

- [Jorge E. Camargo, PhD](https://dis.unal.edu.co/~jecamargom/)

**Asistente docente**:

- [Juan S. Lara MSc](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/)

**Coordinador de virtualización:**

- [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).

**Diseño de imágenes:**
  - [Rosa Alejandra Superlano Esquibel](https://www.linkedin.com/in/alejandra-superlano-02b74313a/).
  - [Mario Andrés Rodríguez Triana](mailto:mrodrigueztr@unal.edu.co).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*